In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import pickle
import sqlalchemy as sa
import datetime as dtt
import calendar
import pytz
import json
from cascade_query import cascade_data

In [188]:
class TotalWorkValue:
    
    def __init__(self, data):
        
        self.dataset = data
        self.default_value = 0.1
        self.df_cascade = pd.read_csv("next_todo.csv").set_index('initial')
        self.df_initial_cals = self._run_initial_calcs()
        
        
    def _run_initial_calcs(self):
        
        df_dataset = self.dataset.copy()
        df_dataset['cc90_rate'] = df_dataset['cc90']/df_dataset['attempt']
        df_dataset['cc90_no_close_rate'] = (df_dataset['cc90']-df_dataset['closed_that_call'])/df_dataset['attempt']
        df_dataset['pgc'] = df_dataset['closed_that_call']/df_dataset['cc90']
        df_dataset['win_rate'] = df_dataset['pgc']*df_dataset['cc90_rate']
        df_dataset['aps'] = df_dataset['purchased_that_call']/df_dataset['closed_that_call']
        df_dataset['talktime_attempt'] = (df_dataset['attempt']*0.5 + (df_dataset['attempt']-df_dataset['cc90'])*1 +\
                                        (df_dataset['cc90']-df_dataset['closed_that_call'])*4+df_dataset['closed_that_call']*15+df_dataset['talktime'])/df_dataset['attempt']
        df_dataset['value_that_call'] = df_dataset['cc90_rate']*df_dataset['pgc']*df_dataset['aps']
        
        df_dataset.fillna(0,inplace  =True)
        
        return(df_dataset)      


    def _value_todo(self,todo_name):

        cascade = self.df_cascade.transpose().to_dict()
        dataset = self.df_initial_cals.set_index('work_type').to_dict()

        value = self.default_value    

        cc90_rate = dataset['cc90_rate']
        cc90_no_close_rate = dataset['cc90_no_close_rate']
        win_rate = dataset['win_rate']
        aps = dataset['aps']
        value_that_call = dataset['value_that_call']
        talktime_attempt = dataset['talktime_attempt']

        no_cc90_next_todo = cascade[todo_name]['nocc90']
        cc90_next_todo = cascade[todo_name]['cc90']

        if(todo_name == '2DFU Ord 2+' or todo_name == '2 Day Leads' or todo_name == 'Golden AI-2+'):

            value = win_rate[todo_name]*aps[todo_name]

            if(value==0):
                value = default_val

        else:
            value = ((win_rate[todo_name])*aps[todo_name])\
                +((cc90_no_close_rate[todo_name])*self._value_todo(cc90_next_todo))\
                +((1-cc90_rate[todo_name])*self._value_todo(no_cc90_next_todo))

        return(value)
    
    def _run_final_calcs(self):
        
        dataset = self.df_initial_cals.copy()
        
        dataset['value_of_todo'] = dataset['work_type'].apply(lambda x: self._value_todo(x))
        dataset['value_of_todo_time'] = dataset['value_of_todo']/dataset['talktime_attempt']
        
        return(dataset[['work_type', 'cc90_rate', 'pgc', 'aps', 'talktime_attempt', 'value_of_todo','value_of_todo_time']])


In [192]:
df_dataset = pd.read_excel("all_data_rd_sep16.xlsx")
rd_list = df_dataset.rd.unique()

df_final = pd.DataFrame()

for rd_val in rd_list:
    
    twv = TotalWorkValue(df_dataset[df_dataset.rd==rd_val])
    
    df_vals = twv._run_final_calcs()
    df_vals['rd'] =  rd_val
    
    df_final = pd.concat([df_final,df_vals])

In [193]:
df_final.to_csv("value_of_work_sep16.csv" , index=False)

In [189]:
twv = TotalWorkValue(df_dataset[df_dataset.rd=='Unscored'])

In [191]:
twv._run_final_calcs()

,work_type,cc90_rate,pgc,aps,talktime_attempt,value_of_todo,value_of_todo_time
1,Viewed Link todo/SCB,0.218954,0.328358,1325.559545,5.538232,113.898962,20.565941
11,Other callbacks,0.184671,0.078369,643.149847,4.636248,29.785375,6.424456
16,No ToDo Outbound,0.323481,0.253162,1059.198822,7.313555,110.131631,15.058563
18,New lead todo/SCB,0.223700,0.048513,884.937258,5.132931,34.845100,6.788539
27,Missed call todo/SCB,0.141096,0.063253,1054.803810,3.625751,27.981130,7.717333
31,M/L priority callbacks,0.080639,0.083841,799.755091,2.695068,20.971222,7.781334
40,Lead action (Regular AI),0.141525,0.270455,1104.640378,3.961423,58.999218,14.893439
42,Lead Action (dep callbacks),0.081220,0.040892,646.454545,2.742665,20.925847,7.629750
53,LFS/SMS Callbacks,0.373006,0.106544,845.995527,8.159502,64.295761,7.879863
57,Inbound (unowned),0.870314,0.091337,804.510855,16.887617,115.906326,6.863392
